#### Import necessary Libraries

In [92]:
import os
import cv2
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import Dense,MaxPool2D,Conv2D,Dropout,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

#### Load csv from directory

In [93]:
datadir=r'C:\Users\raviv\Desktop\archive\PollenDataset\images' # change path as per your directory location

df=pd.read_csv('Honey_bee/pollen_data.csv',)
df.head()

,Unnamed: 0,filename,pollen_carrying
0,0,P10057-125r.jpg,1
1,1,P57380-120r.jpg,1
2,2,NP63671-221r.jpg,0
3,3,NP53776-47r.jpg,0
4,4,NP13382-201r.jpg,0


#### Load image data from directory

In [94]:
image_data=[]
labels=[]

for img,label in zip(df['filename'],df['pollen_carrying']):
    try:
        image=cv2.imread(os.path.join(datadir,img))
        image=cv2.resize(image,dsize=(100,100))
        image_data.append(image)
        labels.append(label)
    
    except Exception as e:
        pass

In [95]:
X=np.array(image_data)
y=np.array(labels)

#### Splita data set into train, test data sets

In [96]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

#### Convert binary labels class into categorical class labels

In [97]:
y_train=to_categorical(y_train,len(np.unique(y_train)))
y_test=to_categorical(y_test,len(np.unique(y_test)))
y_test.shape

(143, 2)

#### Reshape the training and test features

In [98]:
X_train=X_train.reshape(-1,100,100,3)
X_test=X_test.reshape(-1,100,100,3)
X_train.shape

(571, 100, 100, 3)

#### Create Convolutional Neural Network

In [99]:
def cnn_model():
    model=Sequential()
    model.add(Conv2D(filters=64,kernel_size=(2,2),strides=(1,1),padding='valid',input_shape=(100,100,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2),strides=(1,1),padding='valid'))
    
    model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(Conv2D(filters=48,kernel_size=(2,2),strides=(1,1),padding='valid',activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2),strides=(1,1),padding='valid'))
    
    model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2),strides=(1,1),padding='valid'))
    model.add(Conv2D(filters=48,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(Conv2D(filters=48,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(units=200,activation='relu'))
    model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(Flatten())
    
    model.add(Dense(units=2,activation='sigmoid'))
    model.compile(Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
    return model

    

#### Load the defined CNN model

In [100]:
model=cnn_model()

#### Print model summary

In [101]:
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_64 (Conv2D)           (None, 99, 99, 64)        832       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 96, 96, 64)        36928     
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 95, 95, 48)        12336     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 94, 94, 48)        0         
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 92, 92, 64)        27712     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 91, 91, 64)      

#### Train the CNN model on training datasets

In [104]:
h=model.fit(x=X_train,y=y_train,batch_size=15,epochs=30,shuffle=True,validation_split=0.2,verbose=1)

Epoch 1/30
31/31 [==============================] - 13s 407ms/step - loss: 1.1930 - accuracy: 0.5044 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 2/30
31/31 [==============================] - 12s 397ms/step - loss: 0.6929 - accuracy: 0.5373 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 3/30
31/31 [==============================] - 12s 397ms/step - loss: 0.6927 - accuracy: 0.5373 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 4/30
31/31 [==============================] - 12s 397ms/step - loss: 0.6925 - accuracy: 0.5373 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 5/30
31/31 [==============================] - 12s 397ms/step - loss: 0.6923 - accuracy: 0.5373 - val_loss: 0.6931 - val_accuracy: 0.5043
Epoch 6/30
31/31 [==============================] - 12s 397ms/step - loss: 0.6921 - accuracy: 0.5373 - val_loss: 0.6932 - val_accuracy: 0.5043
Epoch 7/30
31/31 [==============================] - 12s 397ms/step - loss: 0.6919 - accuracy: 0.5373 - val_loss: 0.6932 - val_accuracy: 0.5043

#### Evaluate the model and find score

In [116]:
score=model.evaluate(X_test,y_test)
print('test Accuracy is {:.2f} %'.format(score[0]*100))
print('Test score is {:.2f}'.format(score[1]))

5/5 [==============================] - 1s 186ms/step - loss: 0.6995 - accuracy: 0.4615
test Accuracy is 69.95 %
Test score is 0.46


#### Predict the test results

In [117]:
y_predict=np.argmax(model.predict(X_test),axis=1)